In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
csv_file = "../ETL_Project/Bestseller_Publisher.csv"
Bestseller_Publisher_df = pd.read_csv(csv_file)
Bestseller_Publisher_df.head()

,publisher,publisher_id
0,Riverhead,1
1,Scribner,2
2,Vintage,3
3,St. Martin's,4
4,Penguin Group,5


In [3]:
csv_file = "../ETL_Project/Bestseller_Author.csv"
Bestseller_Author_df = pd.read_csv(csv_file)
Bestseller_Author_df.head()

,book_id,publisher_id,author,title,primary_isbn10,date on bestseller,weeks_on_list,book_review_link,sunday_review_link
0,1990,310,A L Jackson,LOST TO YOU,1.938405e+09,3/3/2013,0,NaN,NaN
1,1917,71,A S A Harrison,THE SILENT WIFE,NaN,9/29/2013,0,NaN,NaN
2,1372,38,Aaron Allston,CONVICTION,3.455091e+08,6/12/2011,1,NaN,NaN
3,265,83,Abbi Glines,FALLEN TOO FAR,NaN,3/31/2013,3,NaN,NaN
4,349,83,Abbi Glines,FOREVER TOO FAR,9.883013e+08,7/7/2013,2,NaN,NaN


In [4]:
# Rename Bestseller_Author Column Headers
Bestseller_Author_df = Bestseller_Author_df.rename(columns={'date on bestseller': 'bestseller_date', 
                                                            'weeks_on_list': 'weeks_on_bestseller_list'})
# Select Relevant Columns in new dataframe
New_Bestseller_Author_df = Bestseller_Author_df[['book_id', 'publisher_id','author', 'title', 'bestseller_date', 
                                                 'weeks_on_bestseller_list', 'primary_isbn10']].copy()
New_Bestseller_Author_df.head()

,book_id,publisher_id,author,title,bestseller_date,weeks_on_bestseller_list,primary_isbn10
0,1990,310,A L Jackson,LOST TO YOU,3/3/2013,0,1.938405e+09
1,1917,71,A S A Harrison,THE SILENT WIFE,9/29/2013,0,NaN
2,1372,38,Aaron Allston,CONVICTION,6/12/2011,1,3.455091e+08
3,265,83,Abbi Glines,FALLEN TOO FAR,3/31/2013,3,NaN
4,349,83,Abbi Glines,FOREVER TOO FAR,7/7/2013,2,9.883013e+08


In [5]:
#Connect to Local Database
rds_connection_string = "postgres:postgres@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [6]:
engine.table_names()


['bestseller_publisher', 'bestseller_author']

In [7]:
Bestseller_Publisher_df.to_sql(name='bestseller_publisher', con=engine, if_exists='append', index=False)

In [8]:
New_Bestseller_Author_df.to_sql(name='bestseller_author', con=engine, if_exists='append', index=False)

In [9]:
# Confirm data has been added by querying the bestseller_author table
pd.read_sql_query('select * from bestseller_publisher', con=engine).head()

,publisher_id,publisher
0,1,Riverhead
1,2,Scribner
2,3,Vintage
3,4,St. Martin's
4,5,Penguin Group


In [10]:
#Confirm data has been added by querying the bestseller_author table
pd.read_sql_query('select * from bestseller_author', con=engine).head()

,book_id,publisher_id,author,title,bestseller_date,weeks_on_bestseller_list,primary_isbn10
0,1990,310,A L Jackson,LOST TO YOU,2013-03-03,0,1.938405e+09
1,1917,71,A S A Harrison,THE SILENT WIFE,2013-09-29,0,NaN
2,1372,38,Aaron Allston,CONVICTION,2011-06-12,1,3.455091e+08
3,265,83,Abbi Glines,FALLEN TOO FAR,2013-03-31,3,NaN
4,349,83,Abbi Glines,FOREVER TOO FAR,2013-07-07,2,9.883013e+08


In [11]:

# Confirm Total Count of Publishers
pd.read_sql_query('SELECT COUNT(publisher) As "Total Publishers" FROM bestseller_publisher', con=engine).head()

,Total Publishers
0,327


In [12]:
# Select the average time spend on the bestseller list
pd.read_sql_query('SELECT ROUND(AVG(weeks_on_bestseller_list) ,0) As "Average Weeks on Bestseller List" From bestseller_author', con=engine).head()

,Average Weeks on Bestseller List
0,2.0


In [13]:
# Select the most time spend on the bestseller list
pd.read_sql_query('SELECT Max(weeks_on_bestseller_list) As "Most Weeks Spent on Bestseller List" From bestseller_author', con=engine).head()

,Most Weeks Spent on Bestseller List
0,102


In [14]:
# Select the least time spend on the bestseller list
pd.read_sql_query('SELECT Min(weeks_on_bestseller_list) As "Least Weeks Spent on Bestseller List" From bestseller_author', con=engine).head()

,Least Weeks Spent on Bestseller List
0,0


In [15]:
# Confirm Disntict # Authors
pd.read_sql_query('SELECT Distinct(author) As "Total Authors" FROM bestseller_author',con=engine).head()

,Total Authors
0,Georgia Cates
1,Jeaniene Frost
2,Neil Gaiman
3,Larissa Ione
4,James Patterson with Maxine Paetro


In [17]:
# Joing tables to view publisher, title and author
pd.read_sql_query('SELECT Distinct bestseller_publisher.publisher, bestseller_author.title, bestseller_author.author FROM bestseller_publisher JOIN bestseller_author ON bestseller_publisher.publisher_id = bestseller_author.publisher_id', con=engine).head()

,publisher,title,author
0,Various publishers,SPICE BOX,"Raine Miller, Cathryn Fox and others"
1,Deborah Bladon,RUIN: PART 3,Deborah Bladon
2,Minotaur,NO GOOD DEED,Allison Brennan
3,Kingswell,RAGING HEAT,Richard Castle
4,Morrow/HarperCollins,THE EYE OF GOD,James Rollins
